## Load libraries

In [1]:
# uncomment to install 3D plotting library
# pip install PyGEL3D

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from PyGEL3D import gel
from PyGEL3D import js

import plotly.graph_objects as go

import numpy as np

import sys; sys.path.append("../giotto-learn/") # <--- TMP!!!
from giotto.mapper import *
from giotto.mapper.visualize import create_interactive_network

from sklearn.cluster import DBSCAN, KMeans

import warnings
warnings.filterwarnings("ignore") 

## Load image

In [4]:
m = gel.obj_load("alien.obj")

## Plot image

In [5]:
# plot image and color by height
pos = m.positions()
js.display(m, data=pos[:,2], wireframe=True)

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array(…

## Downsample image

In [24]:
n_samples = 1000 
indices = np.random.choice(pos.shape[0], n_samples, replace=False)  

In [25]:
sample = pos[indices]

In [26]:
x, y, z = sample[:, 0], sample[:, 1], sample[:, 2]

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=1))])
fig.show()

## Rescale to unit height

In [27]:
z_max, z_min = np.max(sample[:, 2]), np.min(sample[:, 2])

In [28]:
scale = z_max - z_min

In [29]:
z_scaled = (z - z_min) / (z_max - z_min)

In [30]:
x_scaled, y_scaled = x / scale, y / scale

In [31]:
fig = go.Figure(data=[go.Scatter3d(x=x_scaled, y=y_scaled, z=z_scaled, mode='markers', marker=dict(size=1))])
fig.show()

## Mapper

The one-dimensional projection does not seem to reveal the desired structure, so project onto x- and z-axis.

In [14]:
X = np.vstack([x_scaled, y_scaled, z_scaled]).T

In [15]:
X.shape

(1000, 3)

In [16]:
# define filter function - can be any scikit-learn Transformer
filter_func = Projection(column_indices=[0,2])
# define cover - only one type is available at the moment
cover = CubicalCover(n_intervals=10, overlap_frac=0.3)
# choose clustering algorithm - default is DBSCAN
clusterer = FirstSimpleGap(linkage='single')

In [17]:
# initialise pipeline
n_jobs_outer = 1

pipe = make_mapper_pipeline(filter_func=filter_func,
                            cover=cover,
                            clusterer=clusterer,
                            verbose=False,
                            n_jobs_outer=n_jobs_outer)

In [18]:
graph = pipe.fit_transform(X)

In [19]:
# OK params: overlap_frac = 0.4 and relative_gap = 0.8
create_interactive_network(pipe, X)

Output()